In [1]:
import os

In [2]:
%pwd

'd:\\PARTH\\Studies\\sem5\\NLC\\project\\text-summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\PARTH\\Studies\\sem5\\NLC\\project\\text-summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [12]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath=CONFIG_FILE_PATH, 
                 params_filepath=PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # FIX: Replaced create_directories with Python's built-in os.makedirs
        os.makedirs(self.config.artifacts_root, exist_ok=True) 

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        # FIX: Replaced create_directories with Python's built-in os.makedirs
        os.makedirs(config.root_dir, exist_ok=True) 

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )

        return data_ingestion_config

In [13]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [14]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(
                f"File: {filename} downloaded with following info: \n{headers}"
            )
        else:
            logger.info(
                f"File already exists of size: {get_size(Path(self.config.local_data_file))}"
            )
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file to the unzip directory
        function returns None
        """
        unzip_path = self.config.unzip_dir

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"File extracted to {unzip_path}")
            
          

In [10]:
!pip install --upgrade ensure

Defaulting to user installation because normal site-packages is not writeable


In [15]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    
    # FIX 1: Pass the config object 'data_ingestion_config'
    # NOT the boolean comparison 'config==data_ingestion_config'
    data_ingestion = DataIngestion(data_ingestion_config) 
    
    data_ingestion.download_file()
    
    # FIX 2: Corrected the typo from 'extract_aip_file' to 'extract_zip_file'
    data_ingestion.extract_zip_file() 

except Exception as e:
    raise e

[2025-10-20 00:22:48,887: INFO: common]: YAML file: config\config.yaml loaded successfully
[2025-10-20 00:22:48,889: INFO: common]: YAML file: params.yaml loaded successfully
[2025-10-20 00:22:52,817: INFO: 740376789]: File: artifacts\data_ingestion\data.zip downloaded with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: A965:33DF8E:323C50:71226E:68F53387
Accept-Ranges: bytes
Date: Sun, 19 Oct 2025 18:52:56 GMT
Via: 1.1 varnish
X-Served-By: cache-bom-vanm7210050-BOM
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1760899976.236503,VS0,VE673
Vary: Authorization,Accept-Encoding
Access-Control-Allow-Origin: *
Cross-Origin-Resource-